In [26]:
from os import path
import re
import pandas as pd
import requests 
import json
import ast

In [7]:
import pandas as pd
corpus_df = pd.read_csv('/home/iftekhar/amiebot/Resources/876/data/corpus_shinzuku_whole.csv')

In [8]:
corpus_df.columns

Index(['page', 'file', 'file_page', 'url', 'title', 'text'], dtype='object')

In [9]:
corpus_df.head()

,page,file,file_page,url,title,text
0,0,NaN,0,https://d.dmkt-sp.jp,dマーケット｜もっと、みつかる、たのしめる。｜NTTドコモ公式,メニュー
1,0,NaN,0,https://d.dmkt-sp.jp,dマーケット｜もっと、みつかる、たのしめる。｜NTTドコモ公式,人気メニュー
2,0,NaN,0,https://d.dmkt-sp.jp,dマーケット｜もっと、みつかる、たのしめる。｜NTTドコモ公式,その他のメニュー
3,0,NaN,0,https://d.dmkt-sp.jp,dマーケット｜もっと、みつかる、たのしめる。｜NTTドコモ公式,dマーケットサービス一覧
4,0,NaN,0,https://d.dmkt-sp.jp,dマーケット｜もっと、みつかる、たのしめる。｜NTTドコモ公式,dアカウント関連メニュー


In [99]:
if 'page' not in corpus_df.columns:
    print('ok')

In [5]:
test_df = pd.read_csv('/home/iftekhar/Documents/test.csv')
urls = test_df.iloc[:, 1:].values.tolist()
questions = test_df['question'].values.tolist()

In [ ]:
urls

In [30]:
questions

['dカーシェアで利用できるカーシェアサービスは何がありますか？',
 'dカーシェアで利用できるカーシェアサービス',
 'カーシェアサービスは何がありますか？',
 'オリックスカーシェア',
 'ｄカーシェアで予約']

In [8]:
corpus_df_url = corpus_df.url.values.tolist()

In [28]:
from collections import defaultdict
test_corpus = defaultdict(list)
count = 0
for url_list in urls:
    for url in url_list:
        # print(url)
        try:
            page_id = list(set(corpus_df[(corpus_df['url'] == url)].page.values))[0]
            test_corpus['page'].append(page_id)
            test_corpus['url'].append(url)
            test_corpus['question'].append(questions[count])
        except IndexError:
            print(url, 'url not found')
    count += 1

NameError: name 'questions' is not defined

In [ ]:
pd.DataFrame(test_corpus)

In [122]:
a=1

In [124]:
b = [a]

### Starting of classification Tag evaluation API

In [41]:
raw_corpus = pd.read_csv('/home/iftekhar/2343/data/corpus.csv')

In [5]:
test_df = pd.read_csv('/home/iftekhar/Downloads/test.csv')

In [6]:
test_df.head()

,url,question,tag
0,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,利用規約に違反した場合、どの様な対応が取られますか？,なし
1,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,料金の未払い,なし
2,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,サービスの利用停止,なし
3,https://faq.carshare.dmkt-sp.jp/faq/show/9?bac...,利用規約に違反,なし
4,https://d.dmkt-sp.jp/common/qa/category/dmarke...,dマーケットとはどのようなサイトですか？,dマーケット


In [10]:
url_list = []
ques = []
ind = 1
collect = []
for index, col in test_df.iterrows():
    url_list.append(col['url'])
    ques.append(col['question'])
    page_id = list(set(corpus_df[(corpus_df['url'] == col['url'])].page.values))
    # print(col['url'], page_id, col['question'], col['tag'])
    if page_id:
        collect.append(page_id[0])
    else:
        collect.append(99999)
    ind += 1

In [ ]:
collect

In [58]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
data = {'output_dir':'/home/iftekhar/2343/model', 
        'data_dir':'/home/iftekhar/2343/data', 
        'agent_id':-1, 
        'query': 'kony mobility platform', 
        'max_answer':5,
        'classification': 'バージョンアップ'
       } 
r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 

# json.loads(r.text)

for i in json.loads(r.text)['page_relevance']:
    print(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0])
    

https://www.softbank.jp/biz/cloud/paas/kony
https://www.softbank.jp/corp/group/sbm/news/press/2015/20150415_02
https://www.softbank.jp/corp/news/press/sbkk/2019/20190416_02


In [61]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
for index, col in test_df.iterrows():
    # data to be sent to api 
    data = {'output_dir':'/home/iftekhar/amiebot/Resources/876/model', 
            'data_dir':'/home/iftekhar/amiebot/Resources/876/data', 
            'agent_id':-1, 
            'query':col['question'], 
            'max_answer':5,
            'classification': col['tag']
           } 
    
    r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 
    result.append(r.text)
    

In [62]:
len(result)

154

In [ ]:
result

In [203]:
r = json.dumps(result)

In [204]:
file = open(path.join('/home/iftekhar/Downloads/', "output.json"), "w", encoding='utf-8')
json.dump(r, file, ensure_ascii=False)
file.close()
    

In [69]:
j = 0
s = 0
for items in saver.values():
    #print(items)
    if items:
        z = [x for x in items.keys()][1:]
        score = mean_reciprocal_rank_score(items['actual'], z)
        if not score > 0:
            s += 1
            print("Not Found: ", ques[j], "\nCorrect", url_list[j])
            for ii in z:
                print('Predict:', ' '.join(list(set(corpus_df[(corpus_df['page'] == ii)].url.values))))
        print(j,':', items['actual'], '->', z, '->', score)
    j += 1
print('Failed ', s )

Not Found:  利用規約に違反した場合、どの様な対応が取られますか？ 
Correct https://faq.carshare.dmkt-sp.jp/faq/show/9?back=front%2Fcategory%3Ashow&category_id=10&page=1&site_domain=default&sort=sort_access&sort_order=descc
Predict: https://faq.carshare.dmkt-sp.jp/print/faq/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/faq/show/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/print/faq/336?site_domain=default
0 : 99999 -> [2123, 1263, 2329] -> 0
Not Found:  料金の未払い 
Correct https://faq.carshare.dmkt-sp.jp/faq/show/9?back=front%2Fcategory%3Ashow&category_id=10&page=1&site_domain=default&sort=sort_access&sort_order=descc
Predict: https://faq.carshare.dmkt-sp.jp/print/faq/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/faq/show/9?site_domain=default
Predict: https://faq.carshare.dmkt-sp.jp/category/show/2?site_domain=default
1 : 99999 -> [2123, 1263, 1024] -> 0
Not Found:  サービスの利用停止 
Correct https://faq.carshare.dmkt-sp.jp/faq/show/9?back=front%2Fcategory%3Ashow&category_

In [100]:
def mean_reciprocal_rank_score(actual_value, predicted_values):
    if not actual_value:
        return 0
    pos = 0
    val = 0
    for i in predicted_values:
        if i == actual_value and pos == 0:
            val = 1
            break
        elif i == actual_value and pos == 1:
            val = 0.5
            break
        elif i == actual_value and pos == 2:
            val = 0.33
            break
        else:
            val = 0
        pos += 1

    return val

In [157]:
test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020_red.csv'
# test_data = '/home/iftekhar/Documents/test_data_docomo_07.12.2020.csv'
test_df = pd.read_csv(test_data)

In [158]:
# defining the api-endpoint  
API_ENDPOINT = "http://127.0.0.1:5000/api/v1.0/predict/"
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
result = []
for index, col in test_df.iterrows():
    # data to be sent to api 
    data = {'output_dir':'/home/iftekhar/docomodezwhu/model', 
            'data_dir':'/home/iftekhar/docomodezwhu/data', 
            'agent_id':-1, 
            'query':col['question'], 
            'max_answer':5
           } 
    
    r = requests.post(url = API_ENDPOINT, data=json.dumps(data), headers = headers) 
    result.append(r.text)
    

In [159]:
raw_corpus = pd.read_csv('/home/iftekhar/docomodezwhu/data/corpus.csv')

In [160]:
from collections import defaultdict
saver = defaultdict(list)

j = 0
for items in result:
    for i in json.loads(items)['page_relevance']:
        saver[j].append(list(set(raw_corpus[raw_corpus['page'] == i['doc_id']].url.values))[0])
        saver[str(str(j)+'doc_id')].append(i['doc_id'])
    j += 1
        

In [161]:
collector = defaultdict(list)
i = 0

for index, col in test_df.iterrows():
    collector['question'].append(col['question'])
    collector['tag'].append(col['tag'])
    collector['correct_url'].append(col['url'])
    collector_correct = []
    try:
        collector_correct = list(set(raw_corpus[raw_corpus['url'] == col['url']].page.values))[0]
    except IndexError:
        continue
        pass
    collector['correct_id'].append(collector_correct)
    collector['returned_id'].append(saver[str(str(i)+'doc_id')])
    collector['mrr_score'].append(mean_reciprocal_rank_score(collector_correct, saver[str(str(i)+'doc_id')]))
    collector['urls'].append(saver[i])
#         print('ok ', i)
    i += 1

    

In [162]:
new_df = pd.DataFrame.from_dict(collector, orient='index').T

In [163]:
new_df.tail()

,question,tag,correct_url,correct_id,returned_id,mrr_score,urls
457,ログインができなくなった,ｄマーケット,https://d.dmkt-sp.jp/common/qa/category/daccou...,None,None,None,None
458,パスワードを忘れた,ｄマーケット,https://d.dmkt-sp.jp/common/qa/category/daccou...,None,None,None,None
459,商品が届かないのですが,ｄマーケット,https://d.dmkt-sp.jp/common/qa/category/other/...,None,None,None,None
460,すごろくのニックネームが既に使われていて登録できません。,ｄマーケット,https://d.dmkt-sp.jp/common/qa/category/sugoro...,None,None,None,None
461,心当たりのない請求がされています。,ｄマーケット,https://d.dmkt-sp.jp/common/qa/category/other/...,None,None,None,None


In [164]:
new_df.to_csv('classification_tag_07.12.2020_part1.csv')